# Teach an LLM to do additions

The goal of this project is to teach an LLM to do additions, playing only with two parts:
* the tokenizer
* the positional embedding

Both the model and the dataset are fixed.

You are allowed to tune the hyperparameters, but this is not the main goal. Depending on the quality of your tokenizer and positional embedding, you may change the number of bits. The initial value of 3 is very small.

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

import random
import math
import re
import time

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
number_bits = 3

dataset_size = 64_000
train_proportion = 0.9

log_interval = 200
batch_size = 64
epochs = 4
learning_rate = 8e-4

## Step 1: Construct a tokenizer

In [5]:
pad_token="[PAD]"
eos_token="[EOS]"

### Baseline: character-level tokenizer

In [6]:
class character_level_tokenizer:
    """
    character-level
    """
    def __init__(self):
        self.vocab = [str(x) for x in range(10)] + ["+", "="] + [pad_token, eos_token]
        self.token_to_id = {v : k for k, v in enumerate(self.vocab)}
        self.id_to_token = {k : v for k, v in enumerate(self.vocab)}
        self.ntokens = len(self.vocab)
        self.pattern = f"[^{re.escape(''.join(self.vocab))}]"
    
    def clean(self, text):
        """
        removes all characters not in the vocabulary
        """
        out = re.sub(self.pattern, "", text)
        return out

    def pre_tokenization(self, text):
        """
        character-level
        """
        return [c for c in text]

    def encode(self, text):
        text_list = self.pre_tokenization(self.clean(text))
        return [self.token_to_id[c] for c in text_list]

    def decode(self, token_list):
        return "".join([self.id_to_token[x] for x in token_list])

In [7]:
tokenizer = character_level_tokenizer()
ntokens = tokenizer.ntokens
ntokens

14

In [8]:
prompt = "12 + 42 ="
inputs = tokenizer.encode(prompt)
inputs, tokenizer.decode(inputs)

([1, 2, 10, 4, 2, 11], '12+42=')

# Implement your tokenizer here!

You can do anything (as long as you do not compute the addition!).
Some ideas:
* reversing numbers left to right
* arranging by groups (of, 2, 3,...)
* aligning numbers

Mon idée est que lorsque l'on faisait des opérations arithmétiques en primaire, on calculait dizaine par dizaine, avec des retenues. On traite donc les chiffres individuellement, mais à la dizaine pertinente. Ainsi, ici, je propose d'encoder chaque chiffre d'un nombre mais en donnant son unité. Par exemple, "123" sera tokénisé en ["100", "20", "3"].

In [9]:
class advanced_tokenizer:
    """
    Encodes numbers as a sequence of digits of variable length depending of their Power of 10
    (e.g. 123 is encoded as 100, 20, 3)
    """
    def __init__(self):
        """Initialises the tokenizer. 
        The vocabulary is all numbers of the form [digit]00...0 , +, =, [PAD], [EOS]
        0 is explicited added to the vocabulary to avoid repetition of the same number"""

        self.vocab = ["0"] + [str(x * 10 ** k) for x in range(1, 10) for k in range(number_bits)] + ["+", "="] + [pad_token, eos_token] 
        self.token_to_id = {v : k for k, v in enumerate(self.vocab)}
        self.id_to_token = {k : v for k, v in enumerate(self.vocab)}
        self.ntokens = len(self.vocab)
        self.pattern = r'[^0-9+=]' # Permet de ne garder que les chiffres, + et = et éviter la fragmentation des nombres à plusieurs chiffres (e.g. 100, 20 ,etc.)
    
    def clean(self, text):
        """
        removes all characters not in the vocabulary
        """
        out = re.sub(self.pattern, "", text)
        return out

    def pre_tokenization(self, text):
        """
        transforms the text into a list of tokens
        a number is a sequence of digits that are encoded with their corresponding number in the vocabulary
        for example, 12 is encoded as 10, 2
        """
        number_length = 0
        text_list = []
        number_digits_list = []
        for c in text:
            if c in self.vocab:
                if c == "0" and number_length == 0:
                    continue # avoid adding 0 at the beginning of a number to not confuse the model
                elif c not in [str(x) for x in range(10)] and number_length == 0: # case for + and =
                    text_list.append(c)
                elif c in [str(x) for x in range(0, 10)]: # if c is a digit
                    number_digits_list.append(c)
                    number_length += 1
                #elif c == "0" and number_length > 0: # if c is a 0 in the middle of a number
                 #   number_length += 1 # avoid to have repeated 0 in the middle of a number
                else : # at the end of a number
                    if number_length > 0:
                        for i in range(number_length):
                            text_list.append(str(10 ** (number_length - i - 1) * int(number_digits_list[i])))
                        number_digits_list = []
                        number_length = 0
                    text_list.append(c)
            else:
                continue

        if number_length > 0: # if the number is at the end of the text (not supposed to happen)
            for i in range(number_length):
                text_list.append(str(10 ** (number_length - i - 1) * int(number_digits_list[i])))


        return text_list


    def encode(self, text):
        text_list = self.pre_tokenization(self.clean(text))
        return [self.token_to_id[c] for c in text_list]

    def decode(self, token_list):
        """Decoder that removes the 0 at the end of a number"""
        output = []
        for x in token_list:
            if x > 9 * number_bits: # if x is not corresponding to a number
                output.append(self.id_to_token[x])
            else:
                output.append(self.id_to_token[x][0])
        return "".join(output)

In [10]:
print("vocabulary: ", advanced_tokenizer().vocab)
print("pretokenisation of 12 + 42 = : ", advanced_tokenizer().pre_tokenization("12 + 42 ="))

vocabulary:  ['0', '1', '10', '100', '2', '20', '200', '3', '30', '300', '4', '40', '400', '5', '50', '500', '6', '60', '600', '7', '70', '700', '8', '80', '800', '9', '90', '900', '+', '=', '[PAD]', '[EOS]']
pretokenisation of 12 + 42 = :  ['10', '2', '+', '40', '2', '=']


In [11]:
tokenizer = advanced_tokenizer()
ntokens = tokenizer.ntokens
ntokens

32

In [12]:
prompt = "12 + 42 ="
inputs = tokenizer.encode(prompt)
inputs, tokenizer.decode(inputs)

([2, 4, 28, 11, 4, 29], '12+42=')

## Step 2: Create a dataset for arithmetic operations

In [13]:
def sample_datapoint(number_bits = 3):
    """
    returns a string containing two random numbers on `number_bits` many bits and their sum.
    """
    a_list = [random.randint(0, 9) for _ in range(number_bits)]
    b_list = [random.randint(0, 9) for _ in range(number_bits)]
    a_int = int("".join([str(x) for x in a_list]))
    b_int = int("".join([str(x) for x in b_list]))
    sum_int = a_int + b_int
    return (str(a_int) + "+" + str(b_int) + "=", str(sum_int))

sample_datapoint(3)

('700+663=', '1363')

In [14]:
data = []
for _ in range(dataset_size):
    data.append(sample_datapoint(number_bits))
data[:4]

[('54+66=', '120'),
 ('119+757=', '876'),
 ('373+609=', '982'),
 ('927+726=', '1653')]

In [15]:
data_train = data[: int(train_proportion * dataset_size)]
data_test = data[int(train_proportion * dataset_size):]

len(data_train),len(data_test)

(57600, 6400)

## Step 3: Construct a model

### Basline: the classical Positional Embedding

In [16]:
class PositionalEmbedding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEmbedder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEmbedder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Implement your positional embedding here!

You can do anything. Some ideas:
* RoPE
* (randomised) FIRE
* Abacus

Pour bien correspondre avec le tokeniseur que nous avons implémenté, nous allons utiliser un abacus. L'idée est d'associer à chaque composante 

Comme dans la baseline, nous utilisons nn.dropout pour éviter une trop grande adaptation des réseaux de neurone (Les raisons de cela sont bien expliquées dans la documentation: https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html).

In [17]:
class AbacusPositionalEmbedding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000, base=10):
        r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        Abacus Positional Embedding
        Args:
        d_model: the embed dim (required);
        dropout: the dropout value (default=0.1);
        max_len: the max. length of the incoming sequence (default=5000);
        base: the base of the positional system (default=10).
        """
        super(AbacusPositionalEmbedding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        self.embed_dim = d_model
        self.base = base
        self.max_digits = len(str(max_len))  # Number of positions to consider
        
        # Embeddings for each power of the base
        self.embedding_layers = nn.ModuleList([
            nn.Embedding(base, d_model) for _ in range(self.max_digits)
        ])
        
    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """
        seq_length, batch_size, embed_dim = x.shape
        embeddings = torch.zeros((seq_length, batch_size, embed_dim), device=x.device)
        
        for i in range(self.max_digits):
            power = self.base ** i
            digit_values = (x // power) % self.base  # Extract significant digits
            embeddings += self.embedding_layers[i](digit_values.long())
        
        return self.dropout(embeddings)


### Transformer model

**!!! IMPORTANT !!!** This model of Transformers is "input first", meaning that an input is a tensor with shape
(length_prompts, batch_size)

In [18]:
class TransformerModel(nn.Transformer):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__(d_model=ninp,
                                               nhead=nhead,
                                               dim_feedforward=nhid,
                                               num_encoder_layers=nlayers)
        self.input_emb = nn.Embedding(ntoken, ninp)
        self.pos_encoder = AbacusPositionalEmbedding(ninp, dropout)
        self.decoder = nn.Linear(ninp, ntoken)

        self.ninp = ninp
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        return torch.log(torch.tril(torch.ones(sz,sz)))

    def forward(self, src):
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        self.src_mask = mask

        src = self.input_emb(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output_enc = self.encoder(src, mask=self.src_mask)
        output_dec = self.decoder(output_enc)
        return F.log_softmax(output_dec, dim=-1), output_enc

Please do not change these parameters!

In [19]:
model = TransformerModel(ntoken = ntokens,
                         ninp = 128,
                         nhead = 16,
                         nhid = 64,
                         nlayers = 8)
model.to(device)

/home/adminialab/miniconda3/envs/llm_env/lib/python3.13/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


TransformerModel(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): Linear(in_features=128, out_features=32, bias=True)
  (input_emb): Embedding(32, 128)
  (pos_encoder): AbacusPositionalEmbedding(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding_layers): M

In [20]:
def generate(model, prompts, new_tokens = 5):
    input_tensor = prompts # (length_prompts, batch_size)
    input_tensor = input_tensor.to(device)
    for _ in range(new_tokens):
        output, _ = model(input_tensor) # (length_prompts, batch_size, ntokens)
        last_output = output[-1,:,:] # (batch_size, ntokens)
        token = torch.argmax(last_output, -1).view((1,-1)) # (1, batch_size)
        input_tensor = torch.cat((input_tensor, token), 0)
    return input_tensor

In [21]:
model.eval()

prompt = "2+3="
prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
output = generate(model, prompt_tensor).view((1,-1))
output, tokenizer.decode(output.tolist()[0])

RuntimeError: output with shape [4, 1, 128] doesn't match the broadcast shape [4, 4, 128, 128]

In [ ]:
def pad(token_list, type_list = "prompts"):
    max_length = max([len(x) for x in token_list])
    out = []
    for x in token_list:
        if type_list == "prompts":
            out.append([tokenizer.token_to_id[pad_token]] * (max_length - len(x)) + x)
        if type_list == "answers":
            out.append(x + [tokenizer.token_to_id[eos_token]] + [tokenizer.token_to_id[pad_token]] * (max_length - len(x)))
    return out, max_length

In [ ]:
prompts = [tokenizer.encode("1+1="), tokenizer.encode("21+35=")]
answers = [tokenizer.encode("2"), tokenizer.encode("56")]
padded_prompts, _ = pad(prompts, "prompts")
padded_answers, _ = pad(answers, "answers")
padded_prompts, padded_answers
[tokenizer.decode(p) for p in padded_prompts], [tokenizer.decode(p) for p in padded_answers]

(['[PAD][PAD]1+1=', '21+35='], ['2[EOS][PAD]', '56[EOS]'])

In [ ]:
def get_batch(split, i):
    data = data_train if split == 'train' else data_test
    prompts = [tokenizer.encode(data[i][0]) for i in range(i, i + batch_size)]
    padded_prompts, length_prompts = pad(prompts, "prompts")
    answers = [tokenizer.encode(data[i][1]) for i in range(i, i + batch_size)]
    padded_answers, length_answers = pad(answers, "answers")
    X = torch.stack([torch.tensor(x) for x in padded_prompts], 1)
    Y = torch.stack([torch.tensor(x) for x in padded_answers], 1)
    return X, Y, length_prompts, length_answers

In [ ]:
X, Y, length_prompts, length_answers = get_batch("train", 243)
X.shape, Y.shape, length_prompts, length_answers

KeyError: '1000'

## Step 4: Evaluate

In [ ]:
def evaluate():
    # Turn on evaluation mode disables dropout.
    model.eval()
    correct = 0.
    with torch.no_grad():
        for batch, i in enumerate(range(0, len(data_test) - 1, batch_size)):
            prompts, target_answers, length_prompts, length_answers = get_batch("test", i)
            prompts = prompts.to(device) # (length_prompts, batch_size)
            target_answers = target_answers.to(device) # (length_answers + 1, batch_size)
            output = generate(model, prompts, length_answers + 1) # (length_prompts + length_answers + 1, batch_size)
            answers_tokens = output[length_prompts:, :] # (length_answers + 1, batch_size), contains tokens
            equality_test = answers_tokens == target_answers # (length_answers + 1, batch_size), contains boolean values
            correct += torch.all(equality_test, axis=0).float().sum()
        accuracy = correct / len(data_test)
    return accuracy.item()

In [ ]:
evaluate()

0.007499999832361937

## Step 4: Train the model

In [ ]:
def train_epoch():
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    total_loss = 0.
    start_time = time.time()
    for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):
        prompts, target_answers, length_prompts, length_answers = get_batch("train", i)
        prompts = prompts.to(device) # (length_prompts, batch_size)
        target_answers = target_answers.to(device) # (length_answers, batch_size)
        input_tensor = torch.cat((prompts, target_answers), 0) # (length_prompts + length_answers, batch_size)
        model.zero_grad()
        output, _ = model(input_tensor) # (length_prompts + length_answers, batch_size, ntokens)
        output_answers = output[length_prompts-1:-1,:,:].reshape(-1, ntokens) # (length_answers * batch_size, ntokens)
        target_answers = target_answers.view(-1)
        loss = F.cross_entropy(output_answers, target_answers)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:5.2f} | perplexity {:8.2f}'.format(batch, len(data_train) // batch_size,
                                                                                                        elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def train():
    best_test_accuracy = None
    test_accuracy = evaluate()
    print('-' * 89)
    print('| initialisation | test accuracy {:5.2f}'.format(test_accuracy))
    print('-' * 89)
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train_epoch()
        test_accuracy = evaluate()
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test accuracy {:5.2f}'.format(epoch, (time.time() - epoch_start_time), test_accuracy))
        print('-' * 89)
        # Save the model if the test accuracy is the best we've seen so far.
        if not best_test_accuracy or test_accuracy < best_test_accuracy:
            with open("arithmetic.pt", 'wb') as f:
                torch.save(model, f)
            best_test_accuracy = test_accuracy

In [ ]:
train()

-----------------------------------------------------------------------------------------
| initialisation | test accuracy  0.01
-----------------------------------------------------------------------------------------
|   200/  900 batches | ms/batch 17.23 | loss  0.03 | perplexity     1.03
|   400/  900 batches | ms/batch 16.77 | loss  0.00 | perplexity     1.00
|   600/  900 batches | ms/batch 16.74 | loss  0.00 | perplexity     1.00
|   800/  900 batches | ms/batch 16.97 | loss  0.00 | perplexity     1.00
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 15.80s | test accuracy  1.00
-----------------------------------------------------------------------------------------
|   200/  900 batches | ms/batch 16.82 | loss  0.00 | perplexity     1.00
|   400/  900 batches | ms/batch 16.83 | loss  0.00 | perplexity     1.00
|   600/  900 batches | ms/batch 16.66 | loss  0.00 | perplexity     1.00
|   800/  900 batches | ms/

In [ ]:
model.eval()

for i in range(20):
    prompt, answers = data_test[i]
    prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
    output = generate(model, prompt_tensor, len(answers)).view((1,-1))
    print(tokenizer.decode(output.tolist()[0]) + "\t actual result: " + answers)

166+642=[EOS][EOS][EOS]	 actual result: 808
535+994=[EOS][EOS][EOS][EOS]	 actual result: 1529
240+537=[EOS][EOS][EOS]	 actual result: 777
723+179=[EOS][EOS][EOS]	 actual result: 902
146+951=[EOS][EOS][EOS][EOS]	 actual result: 1097
712+269=[EOS][EOS][EOS]	 actual result: 981
712+105=[EOS][EOS][EOS]	 actual result: 817
923+522=[EOS][EOS][EOS][EOS]	 actual result: 1445
447+445=[EOS][EOS][EOS]	 actual result: 892
583+712=[EOS][EOS][EOS][EOS]	 actual result: 1295
819+5=[EOS][EOS][EOS]	 actual result: 824
388+715=[EOS][EOS][EOS][EOS]	 actual result: 1103
763+225=[EOS][EOS][EOS]	 actual result: 988
439+871=[EOS][EOS][EOS][EOS]	 actual result: 1310
843+146=[EOS][EOS][EOS]	 actual result: 989
998+563=[EOS][EOS][EOS][EOS]	 actual result: 1561
125+12=[EOS][EOS][EOS]	 actual result: 137
563+452=[EOS][EOS][EOS][EOS]	 actual result: 1015
311+99=[EOS][EOS][EOS]	 actual result: 410
835+794=[EOS][EOS][EOS][EOS]	 actual result: 1629


## Probing

This is just for fun...

In [ ]:
import numpy as np

train_size = 1000
test_size = 100

model.eval()

def data_probing(size):
    X = []
    y = np.zeros(size)
    for i in range(size):
        input = torch.tensor(tokenizer.encode(data[i][0])).view((-1, 1)).to(device)
        _, output = model(input)
        output = output[-1,:,:].flatten()
        # determine whether there was a carry in the result:
        carry = len(data[i][1]) > len(data[i][0]) / 2
        X.append(output.cpu().detach().numpy())
        y[i] = carry
    return np.array(X), y

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X_train, y_train = data_probing(train_size)
X_test, y_test = data_probing(test_size)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

reg = LogisticRegression()
reg.fit(X_train,y_train)
reg.score(X_test, y_test)

1.0